In [4]:
import numpy as np
import pandas as pd

------
## Simplificando as perguntas
--------

In [5]:
vet = pd.read_csv('VETERANOS.csv')\
    .dropna(axis=1, how='all')\
    .dropna(axis=0, how='all')\
    .drop('Você é ou já foi aluno do curso de Ciência da Computação ou Engenharia da Computação?', 1)\
    .drop('Unnamed: 0', 1)\
    .drop('Unnamed: 18', 1)\
    .fillna('')

------
## Funções Utilizadas
--------

Neste notebook, foram utilizadas as seguintes funções para a contagem das categorias e suas cross-relações:
- cross table
- cross table com dados relativos (%)

Abaixo estão outras funções auxiliares

#### IMPORTANTE: por favor, inicialize as variáveis antes de usar as funções

In [6]:
# df.corr(method ='pearson')  # correlação de pearson para variaveis quantitativas
# pd.crosstab(df['col1'], df['col2']) # matriz de confusão do python

# the same as split
def separate(s, sep):
    return str(s).split(sep)



# stacks two lists in vertical - must have the same columns size
def append(list1, list2):
    res = list1.copy()
    for l in list2:
        res.append(l)
    return res



# stacks two pandas DataFrame in vertical - must have the same columns size. The columns names are passed by argument
def appendDataFrames(df1, df2, columns):
    l1 = df1.values.tolist()
    l2 = df2.values.tolist()
    
    return pd.DataFrame(append(l1,l2), columns=columns)



# splits categories jointed by ''/''
def disjoint(df, col):
    res = []
    indexes = ["/" not in i for i in df[col]]
    newDF = df.loc[indexes]
    
    for index, row in df.iterrows():
        if("/" in row[col]):
            [c1, c2] = row[col].split(" / ")
            
            newRow1 = row.copy()
            newRow1[col] = c1
            
            newRow2 = row.copy()
            newRow2[col] = c2            
            
            res.append(newRow1)
            res.append(newRow2)
            
    return appendDataFrames(newDF, pd.DataFrame(res, columns=newDF.columns, index=[len(df.values.tolist()) + i for i in range(0, len(res))]), columns=df.columns)





# returns cross table - yet this already exists in python
def countCols(table, col1, col2):
    # create dictionary
    df = pd.DataFrame(table)
    
    # the counter dictionary
    counts = {}
    
    for index, row in df.iterrows():
        # separating, if needed
        rowContents1 = separate(row[col1], " / ")
        rowContents2 = separate(row[col2], " / ")
        
        for r1 in rowContents1:
            for r2 in rowContents2:
                # making the key
                k = r1 + "_" + r2
                
                # test if the key already exists
                if k not in counts:
                    counts[k] = 0
                counts[k] = int(counts[k] + 1)
        
    # count #rows and #cols
    rows = []
    cols = []
    for key in counts.keys():
        #separating row and col
        [rowIndex, colIndex] = separate(key, "_")
        
        # testing if the indexes is not included yet
        if(not np.isin([rowIndex], rows)[0]):
            rows.append(rowIndex)
        if(not np.isin([colIndex], cols)[0]):
            cols.append(colIndex)
    
    # create new table
    a = np.zeros(shape=(len(rows), len(cols)), dtype=int)
    finalTable = pd.DataFrame(a,columns=cols, index=rows)
    
    # inserting into the final table
    for key in counts.keys():
        [rowIndex, colIndex] = separate(key, "_")
        finalTable[colIndex][rowIndex] = int(counts[rowIndex + "_" + colIndex])
        
    return finalTable


# replaces the category codes by their category names 
def switchBack(df, col):
    # using the variable codesByCat defined below
    res = []
    for index, row in df.iterrows():
        newRow1 = row.copy()
        
        # separating the categories, if so
        if ("/" in row[col]):
            [cat1, cat2] = row[col].split(" / ")
            
            # removing possible whitespaces
            cat1 = cat1.strip()
            cat2 = cat2.strip()
            
            # testing if this is a valid code
            if(cat1 not in codesByCat[col].keys() or cat2 not in codesByCat[col].keys()):
                print("Removendo outlier:", index, row[col], "++++++++")
                continue
            newRow1[col] = codesByCat[col][cat1] + " / " + codesByCat[col][cat2]
        elif(row[col] != ""):
            # testing if this is a valid code
            if(row[col] not in codesByCat[col].keys()):
                print("Removendo outlier:", index, row[col], "++++++++")
                continue

            newRow1[col] = codesByCat[col][row[col].strip()]
        
        # putting the new row on the bottom of matrix
        res.append(newRow1)
    
    return pd.DataFrame(res, columns=df.columns)


--------
## Mostrando a Tabela de VETERANOS
--------

-------
## Tabela dos Veteranos - Dados Brutos
-------

Aqui, estão apresentados os dados com as categorias codificadas, tendo como base a análise de conteúdo realizada. 

In [10]:
# renaming questions to simplified names

vet = vet.rename(columns=map_col_vet)

vet

,idade,gênero,curso,instituição,semestre entrada,motivação para entrar,expectativas,mercado de trabalho,expectativas atendidas,dificuldades,dificuldades podem te fazer desistir?,quais?,uso de jogos impacta na visão sobre o curso?
0,16 a 19,Feminino,Ciência da Computação,Insituto Federal do Sul de Minas Gerais,2017.1,Emp / IntAC,CCMat / PAMasc,,4,DOrgP,"Não, de jeito nenhum!",,4
1,24 a 27,Feminino,Ciência da Computação,UECE,2016.2,IntAC,CCDif / PAEnt,BEmp,3,DidProf / ProbInf,"Não, de jeito nenhum!",,5
2,24 a 27,Feminino,Ciência da Computação,UFC,2015.1,IntAC,CCPrat,,2,DidProf / EstrCInf,"Sim, com certeza!",,5
3,20 a 23,Feminino,Engenharia da Computação,UFC,2016.1,IntAC,CapAp,TrabAI,4,DifDisc / BConR,"Sim, com certeza!",BConR,2
4,24 a 27,Feminino,Ciência da Computação,UFC,2016.2,IntAC,EP,,4,DifDisc,"Não, de jeito nenhum!",,3
5,16 a 19,Feminino,Ciência da Computação,UFC,2016.1,IntAC,CCProg,,3,DifDisc,"Não, de jeito nenhum!",,4
6,20 a 23,Feminino,Engenharia da Computação,UFC,2016.1,IntAC,AtivEC,BEmp,1,EstrCInf / ExCHO,"Sim, com certeza!",EstrCInf,4
7,16 a 19,Masculino,Ciência da Computação,Ifnmg,2018.1,IntAC,CapAp,EN,5,ExCHO,"Sim, com certeza!",DifFin,1
8,28 ou mais,Masculino,Ciência da Computação,UFC,2012.1,Emp / IntAC,CCMerc,EP,4,ExCHO,"Sim, com certeza!",HorConf / DifFin,3
9,24 a 27,Masculino,Ciência da Computação,UFC,2014.1,Emp / IntAC,EP,EP,1,DiscAM,"Sim, com certeza!",DifDisc,3


---------
## Tabela de Veteranos - Categorias por extenso
--------

Neste trecho, os códigos das categorias são trocadas pelos nomes completos, a fim de melhorar a compreensão das mesmas.

In [11]:

# trocando os codigos pela categoria por extenso

vet_extenso = switchBack(vet, "motivação para entrar")
vet_extenso = switchBack(vet_extenso, "expectativas")
vet_extenso = switchBack(vet_extenso, "mercado de trabalho")
vet_extenso = switchBack(vet_extenso, "dificuldades")
vet_extenso = switchBack(vet_extenso, "quais?")

vet_extenso
    

Removendo outlier: 10 OUTLIER ++++++++


,idade,gênero,curso,instituição,semestre entrada,motivação para entrar,expectativas,mercado de trabalho,expectativas atendidas,dificuldades,dificuldades podem te fazer desistir?,quais?,uso de jogos impacta na visão sobre o curso?
0,16 a 19,Feminino,Ciência da Computação,Insituto Federal do Sul de Minas Gerais,2017.1,Empregabilidade / Interesse por áreas do curso,Curso com caráter matemático / Perfil majorita...,,4,Dificuldades de foco e organização pessoal,"Não, de jeito nenhum!",,4
1,24 a 27,Feminino,Ciência da Computação,UECE,2016.2,Interesse por áreas do curso,Curso com caráter difícil / Perfil de alunos e...,Boa empregabilidade,3,Didática dos professores / Problemas na infrae...,"Não, de jeito nenhum!",,5
2,24 a 27,Feminino,Ciência da Computação,UFC,2015.1,Interesse por áreas do curso,Curso com caráter prático,,2,Didática dos professores / Estrutura do curso ...,"Sim, com certeza!",,5
3,20 a 23,Feminino,Engenharia da Computação,UFC,2016.1,Interesse por áreas do curso,Capacitação e aprendizado,Trabalhar na área de interesse,4,Dificuldade e exigência das disciplinas / Base...,"Sim, com certeza!",Base pessoal de conhecimento ruim,2
4,24 a 27,Feminino,Ciência da Computação,UFC,2016.2,Interesse por áreas do curso,Expectativas positivas,,4,Dificuldade e exigência das disciplinas,"Não, de jeito nenhum!",,3
5,16 a 19,Feminino,Ciência da Computação,UFC,2016.1,Interesse por áreas do curso,Curso com caráter voltador para programação,,3,Dificuldade e exigência das disciplinas,"Não, de jeito nenhum!",,4
6,20 a 23,Feminino,Engenharia da Computação,UFC,2016.1,Interesse por áreas do curso,Atividades extracurriculares,Boa empregabilidade,1,Estrutura do curso inflexível / Excesso de car...,"Sim, com certeza!",Estrutura do curso inflexível,4
7,16 a 19,Masculino,Ciência da Computação,Ifnmg,2018.1,Interesse por áreas do curso,Capacitação e aprendizado,Expectativas negativas,5,Excesso de carga horária obrigatória,"Sim, com certeza!",Dificuldades financeiras,1
8,28 ou mais,Masculino,Ciência da Computação,UFC,2012.1,Empregabilidade / Interesse por áreas do curso,Curso com caráter mercadológico,Expectativas positivas,4,Excesso de carga horária obrigatória,"Sim, com certeza!",Horários conflitantes / Dificuldades financeiras,3
9,24 a 27,Masculino,Ciência da Computação,UFC,2014.1,Empregabilidade / Interesse por áreas do curso,Expectativas positivas,Expectativas positivas,1,Discrepância entre academia e mercado de trabalho,"Sim, com certeza!",Dificuldade e exigência das disciplinas,3


----------
## Tabela dos Veteranos - categorias separadas
----------

Aqui, nos deparamos com o seguinte problema: quando realizamos a contagem das categorias, algumas respostas apresentavam mais de uma delas. 
\Com isso, o script entendia que a resposta composta era uma nova categoria e a contabilizava individualmente e isto é indesejado

Logo, precisamos dividí-las, colocando-as em linhas separadas (onde cada linha representa uma resposta) e replicando o resto da linha.

Daí, vem a preocupação de estarmos replicando dados de forma artificial, mas contornamos da seguinte forma:
- Se a coluna teve respostas compostas (i.e., que tiveram que ser separadas), então a contagem dela só pode acontecer individualmente ou relacionada a outra coluna de mesmo tipo (e.g., motivação e dificuldade)
- Caso contrário, a contagem acontece na tabela original, sem separação.

As perguntas que tiveram respostas compostas foram:
* Motivação para entrar no curso
* expectativas para com o curso
* dificuldades enfrentadas
* expectativas sobre o mercado de trabalho

Abaixo é mostrado o código, utilizando a função disjoint, e a tabela corrigida.

In [12]:
vet_separados = disjoint(vet_extenso, "quais?")

vet_separados = disjoint(vet_separados, "motivação para entrar")

vet_separados = disjoint(vet_separados, "expectativas")

vet_separados = disjoint(vet_separados, "dificuldades")

vet_separados = disjoint(vet_separados, "mercado de trabalho")

vet_separados


,idade,gênero,curso,instituição,semestre entrada,motivação para entrar,expectativas,mercado de trabalho,expectativas atendidas,dificuldades,dificuldades podem te fazer desistir?,quais?,uso de jogos impacta na visão sobre o curso?
0,24 a 27,Feminino,Ciência da Computação,UFC,2016.2,Interesse por áreas do curso,Expectativas positivas,,4,Dificuldade e exigência das disciplinas,"Não, de jeito nenhum!",,3
1,16 a 19,Feminino,Ciência da Computação,UFC,2016.1,Interesse por áreas do curso,Curso com caráter voltador para programação,,3,Dificuldade e exigência das disciplinas,"Não, de jeito nenhum!",,4
2,16 a 19,Masculino,Ciência da Computação,Ifnmg,2018.1,Interesse por áreas do curso,Capacitação e aprendizado,Expectativas negativas,5,Excesso de carga horária obrigatória,"Sim, com certeza!",Dificuldades financeiras,1
3,20 a 23,Masculino,Ciência da Computação,UFC,2015.1,Interesse por áreas do curso,Curso com caráter acadêmico,Nenhuma,5,Dificuldade e exigência das disciplinas,"Não, de jeito nenhum!",,5
4,24 a 27,Masculino,Ciência da Computação,UFC,2015.1,Interesse por áreas do curso,Curso com caráter prático,Diversidade de oportunidades,3,Base pessoal de conhecimento ruim,"Não, de jeito nenhum!",Não sei,5
5,20 a 23,Masculino,Ciência da Computação,UFC,2016.1,Interesse por áreas do curso,Curso com caráter menos matemático,Expectativas positivas,4,Base pessoal de conhecimento ruim,"Não, de jeito nenhum!",,3
6,16 a 19,Masculino,Engenharia da Computação,UFC,2017.1,Interesse por áreas do curso,Capacitação e aprendizado,Empregabilidade regional ruim,2,Discrepância entre academia e mercado de trabalho,"Sim, com certeza!",,3
7,20 a 23,Masculino,Engenharia da Computação,UFC,2017.1,Empregabilidade,Projetos práticos,Discrepância entre academia e mercado,2,Discrepância entre academia e mercado de trabalho,"Sim, com certeza!",Dificuldade e exigência das disciplinas,4
8,20 a 23,Masculino,Engenharia da Computação,UFC,2017.1,Interesse por áreas do curso,Expectativas positivas,Trabalhar na área de interesse,4,Discrepância entre academia e mercado de trabalho,"Sim, com certeza!",Dificuldades financeiras,4
9,16 a 19,Masculino,Ciência da Computação,UFC,2018.1,Interesse por áreas do curso,Capacitação e aprendizado,Expectativas positivas,5,Dificuldades de foco e organização pessoal,"Não, de jeito nenhum!",,3


--------
## Contagens
--------

#### IMPORTANTE
Sempre que a função lidar com a tabela "vet_extenso", significa que os dados são da tabela original, sem separação de respostas compostas.

Já quando utiliza a tabela "vet_separados", os dados foram tratados com relação a respostas compostas (i.e., a resposta ocupa mais de uma categoria da análise de conteúdo)

### Curso

In [13]:
vet_extenso['curso'].value_counts()

Ciência da Computação       23
Engenharia da Computação    11
Name: curso, dtype: int64

----------
### Faixa Etária

In [8]:
vet_extenso['idade'].value_counts()

20 a 23       14
16 a 19       13
24 a 27        5
28 ou mais     2
Name: idade, dtype: int64

----------
### Gênero

In [9]:
vet_extenso['gênero'].value_counts()

Masculino    27
Feminino      7
Name: gênero, dtype: int64

----------
### Motivação para entrar


In [14]:
vet_separados['motivação para entrar'].value_counts()

Interesse por áreas do curso    45
Empregabilidade                 11
Influência de Terceiros          4
Name: motivação para entrar, dtype: int64

--------
### Expectativas

In [15]:
vet_separados['expectativas'].value_counts()

Capacitação e aprendizado                      30
Expectativas positivas                          5
Curso com caráter mercadológico                 4
Curso com caráter prático                       3
Curso com caráter difícil                       3
Perfil majoritariamente masculino               2
Perfil de alunos entusiasmados                  2
Curso com caráter matemático                    2
Bom networking                                  2
Atividades extracurriculares                    2
Expectativas medianas                           1
Curso com caráter menos matemático              1
Curso com caráter voltador para programação     1
Projetos práticos                               1
Curso com caráter acadêmico                     1
Name: expectativas, dtype: int64

--------
### Expectativas Atendidas

In [12]:
pd.DataFrame(vet_extenso['expectativas atendidas'].replace("", "Não sabem / Não opinaram").value_counts()).sort_index(ascending=False)

,expectativas atendidas
5,3
4,14
3,10
2,5
1,2


--------
### Expectativas Atendidas X Expectativas

In [17]:
countCols(vet_separados,  "expectativas atendidas", "expectativas").sort_index(ascending=False)

,Expectativas positivas,Curso com caráter voltador para programação,Capacitação e aprendizado,Curso com caráter acadêmico,Curso com caráter prático,Curso com caráter menos matemático,Projetos práticos,Expectativas medianas,Curso com caráter mercadológico,Curso com caráter difícil,Curso com caráter matemático,Perfil majoritariamente masculino,Atividades extracurriculares,Bom networking,Perfil de alunos entusiasmados
5,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0
4,2,0,13,0,0,1,0,0,4,1,2,2,0,2,0
3,0,1,10,0,1,0,0,1,0,2,0,0,0,0,2
2,1,0,5,0,2,0,1,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0


--------
### Mercado de Trabalho

In [18]:
vet_separados['mercado de trabalho'].replace("", "Não sabem / Não opinaram").value_counts()

Expectativas positivas                   15
Boa empregabilidade                      11
Diversidade de oportunidades              9
Não sabem / Não opinaram                  8
Empregabilidade regional ruim             6
Nenhuma                                   3
Trabalhar na área de interesse            3
Mercado concorrido                        2
Discrepância entre academia e mercado     1
Expectativas negativas                    1
Socialmente Relevante                     1
Name: mercado de trabalho, dtype: int64

--------
### Dificuldades Enfrentadas

In [19]:
vet_separados['dificuldades'].replace("", "Não sabem / Não opinaram").value_counts()

Dificuldade e exigência das disciplinas              15
Excesso de carga horária obrigatória                  9
Dificuldades de foco e organização pessoal            6
Didática dos professores                              5
Discrepância entre academia e mercado de trabalho     5
Base pessoal de conhecimento ruim                     4
Estrutura do curso inflexível                         4
Dificuldades financeiras                              4
Falta de motivação pessoal                            3
Problemas na infraestrutura do campus                 2
Perfil de alunos do curso                             2
Horários conflitantes                                 1
Name: dificuldades, dtype: int64

--------
### Dificuldades Fariam Desistir?

In [13]:
vet_extenso['dificuldades podem te fazer desistir?'].replace("", "Não sabem / Não opinaram").value_counts()

Não, de jeito nenhum!    21
Sim, com certeza!        13
Name: dificuldades podem te fazer desistir?, dtype: int64

--------
### Quais?

In [21]:
vet_separados['quais?'].replace("", "Não desistiriam / campo vazio").value_counts()

Não desistiriam / campo vazio              38
Dificuldade e exigência das disciplinas     6
Estrutura do curso inflexível               4
Dificuldades financeiras                    4
Base pessoal de conhecimento ruim           2
Horários conflitantes                       2
Falta de motivação pessoal                  2
Didática dos professores                    1
Não sei                                     1
Name: quais?, dtype: int64

--------
### Dificuldades que fariam desistir ou Não

In [22]:
countCols(vet_separados,  "dificuldades podem te fazer desistir?", "dificuldades")

,Dificuldade e exigência das disciplinas,Excesso de carga horária obrigatória,Base pessoal de conhecimento ruim,Discrepância entre academia e mercado de trabalho,Dificuldades de foco e organização pessoal,Dificuldades financeiras,Didática dos professores,Estrutura do curso inflexível,Horários conflitantes,Falta de motivação pessoal,Perfil de alunos do curso,Problemas na infraestrutura do campus
"Não, de jeito nenhum!",13,2,3,0,6,4,2,1,1,0,0,2
"Sim, com certeza!",2,7,1,5,0,0,3,3,0,3,2,0


--------
### Dificuldades enfrentadas X Quais? (melhorar)

In [14]:
dif_by_quais = countCols(vet_separados,  "dificuldades", "quais?")
dif_by_quais.columns = [ "Não desistiria / campo vazio" if (i == "") else i for i in dif_by_quais.columns]
dif_by_quais

,Não desistiria / campo vazio,Dificuldades financeiras,Não sei,Dificuldade e exigência das disciplinas,Didática dos professores,Horários conflitantes,Base pessoal de conhecimento ruim,Estrutura do curso inflexível,Falta de motivação pessoal
Dificuldade e exigência das disciplinas,14,0,0,0,0,0,1,0,0
Excesso de carga horária obrigatória,2,3,0,0,0,2,0,2,0
Base pessoal de conhecimento ruim,2,0,1,0,0,0,1,0,0
Discrepância entre academia e mercado de trabalho,1,1,0,3,0,0,0,0,0
Dificuldades de foco e organização pessoal,6,0,0,0,0,0,0,0,0
Dificuldades financeiras,4,0,0,0,0,0,0,0,0
Didática dos professores,3,0,0,1,1,0,0,0,0
Estrutura do curso inflexível,2,0,0,0,0,0,0,2,0
Horários conflitantes,1,0,0,0,0,0,0,0,0
Falta de motivação pessoal,1,0,0,1,0,0,0,0,1


## Gênero x curso

* ### contagem absoluta

In [16]:

tab_conting_veteranos = pd.crosstab(vet_extenso['gênero'], vet_extenso['curso'])
tab_conting_veteranos

curso,Ciência da Computação,Engenharia da Computação
gênero,,
Feminino,5,2
Masculino,18,9


----------
* ### contagem relativa (%)

In [17]:
tab_conting_veteranos.apply(lambda r: (r)/r.sum(), axis=1).style.format("{:.2%}") # formats 2 digits percentage

curso,Ciência da Computação,Engenharia da Computação
gênero,,
Feminino,71.43%,28.57%
Masculino,66.67%,33.33%


----------
## Curso x Motivação para entrar

----------
* ### contagem absoluta

In [26]:
tab_cross_motiv = pd.crosstab(vet_separados['curso'], vet_separados['motivação para entrar'])
tab_cross_motiv

motivação para entrar,Empregabilidade,Influência de Terceiros,Interesse por áreas do curso
curso,,,
Ciência da Computação,9,2,31
Engenharia da Computação,2,2,14


----------
* ### contagem relativa (%)

In [27]:
tab_cross_motiv.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

motivação para entrar,Empregabilidade,Influência de Terceiros,Interesse por áreas do curso
curso,,,
Ciência da Computação,21.43%,4.76%,73.81%
Engenharia da Computação,11.11%,11.11%,77.78%


## Curso x Expectativas

----------
* ### contagem absoluta

In [29]:
tab_cross_expec = pd.crosstab(vet_separados['curso'], vet_separados['expectativas'])
tab_cross_expec

expectativas,Atividades extracurriculares,Bom networking,Capacitação e aprendizado,Curso com caráter acadêmico,Curso com caráter difícil,Curso com caráter matemático,Curso com caráter menos matemático,Curso com caráter mercadológico,Curso com caráter prático,Curso com caráter voltador para programação,Expectativas medianas,Expectativas positivas,Perfil de alunos entusiasmados,Perfil majoritariamente masculino,Projetos práticos
curso,,,,,,,,,,,,,,,
Ciência da Computação,0,2,17,1,2,2,1,4,3,1,1,4,2,2,0
Engenharia da Computação,2,0,13,0,1,0,0,0,0,0,0,1,0,0,1


----------
* ### contagem relativa (%)

In [30]:
tab_cross_expec.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

expectativas,Atividades extracurriculares,Bom networking,Capacitação e aprendizado,Curso com caráter acadêmico,Curso com caráter difícil,Curso com caráter matemático,Curso com caráter menos matemático,Curso com caráter mercadológico,Curso com caráter prático,Curso com caráter voltador para programação,Expectativas medianas,Expectativas positivas,Perfil de alunos entusiasmados,Perfil majoritariamente masculino,Projetos práticos
curso,,,,,,,,,,,,,,,
Ciência da Computação,0.00%,4.76%,40.48%,2.38%,4.76%,4.76%,2.38%,9.52%,7.14%,2.38%,2.38%,9.52%,4.76%,4.76%,0.00%
Engenharia da Computação,11.11%,0.00%,72.22%,0.00%,5.56%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,5.56%,0.00%,0.00%,5.56%


## Curso x Expectativas atendidas

----------
* ### contagem absoluta

In [18]:
tab_cross_atend = pd.crosstab(vet_extenso['curso'], vet_extenso['expectativas atendidas'])
tab_cross_atend

expectativas atendidas,1,2,3,4,5
curso,,,,,
Ciência da Computação,1,3,8,8,3
Engenharia da Computação,1,2,2,6,0


----------
* ### contagem relativa (%)

In [19]:
tab_cross_atend.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

expectativas atendidas,1,2,3,4,5
curso,,,,,
Ciência da Computação,4.35%,13.04%,34.78%,34.78%,13.04%
Engenharia da Computação,9.09%,18.18%,18.18%,54.55%,0.00%


---------------
## Curso x Dificuldades

----------
* ### contagem absoluta

In [33]:
tab_cross_dif = pd.crosstab(vet_separados['curso'], vet_separados['dificuldades'])
tab_cross_dif

dificuldades,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades de foco e organização pessoal,Dificuldades financeiras,Discrepância entre academia e mercado de trabalho,Estrutura do curso inflexível,Excesso de carga horária obrigatória,Falta de motivação pessoal,Horários conflitantes,Perfil de alunos do curso,Problemas na infraestrutura do campus
curso,,,,,,,,,,,,
Ciência da Computação,3,5,8,6,4,2,1,6,2,1,2,2
Engenharia da Computação,1,0,7,0,0,3,3,3,1,0,0,0


----------
* ### contagem relativa (%)

In [34]:
tab_cross_dif.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

dificuldades,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades de foco e organização pessoal,Dificuldades financeiras,Discrepância entre academia e mercado de trabalho,Estrutura do curso inflexível,Excesso de carga horária obrigatória,Falta de motivação pessoal,Horários conflitantes,Perfil de alunos do curso,Problemas na infraestrutura do campus
curso,,,,,,,,,,,,
Ciência da Computação,7.14%,11.90%,19.05%,14.29%,9.52%,4.76%,2.38%,14.29%,4.76%,2.38%,4.76%,4.76%
Engenharia da Computação,5.56%,0.00%,38.89%,0.00%,0.00%,16.67%,16.67%,16.67%,5.56%,0.00%,0.00%,0.00%


---------------
## Curso x dificuldades podem te fazer desistir?

----------
* ### contagem absoluta

In [20]:
tab_cross_dif_giv = pd.crosstab(vet_extenso['curso'], vet_extenso['dificuldades podem te fazer desistir?'])
tab_cross_dif_giv

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
curso,,
Ciência da Computação,17,6
Engenharia da Computação,4,7


----------
* ### contagem relativa (%)

In [21]:
tab_cross_dif_giv.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
curso,,
Ciência da Computação,73.91%,26.09%
Engenharia da Computação,36.36%,63.64%


---------------
## Curso x QUAIS dificuldades podem te fazer desistir?

----------
* ### contagem absoluta

In [23]:
tab_cross_dif_quais = pd.crosstab(vet_separados['curso'], vet_separados['quais?'])
tab_cross_dif_quais.columns = [ "Não desistiria / campo vazio" if (i == "") else i for i in tab_cross_dif_quais.columns]
tab_cross_dif_quais

,Não desistiria / campo vazio,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades financeiras,Estrutura do curso inflexível,Falta de motivação pessoal,Horários conflitantes,Não sei
curso,,,,,,,,,
Ciência da Computação,28,0,1,5,3,0,2,2,1
Engenharia da Computação,10,2,0,1,1,4,0,0,0


----------
* ### contagem relativa (%)

In [24]:
tab_cross_dif_quais.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

,Não desistiria / campo vazio,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades financeiras,Estrutura do curso inflexível,Falta de motivação pessoal,Horários conflitantes,Não sei
curso,,,,,,,,,
Ciência da Computação,66.67%,0.00%,2.38%,11.90%,7.14%,0.00%,4.76%,4.76%,2.38%
Engenharia da Computação,55.56%,11.11%,0.00%,5.56%,5.56%,22.22%,0.00%,0.00%,0.00%


---------------
## Curso x uso de jogos impacta na visão sobre o curso?

----------
* ### contagem absoluta

In [25]:
tab_cross_dif_jogos = pd.crosstab(vet_separados['curso'], vet_separados['uso de jogos impacta na visão sobre o curso?'])
tab_cross_dif_jogos

uso de jogos impacta na visão sobre o curso?,1,2,3,4,5
curso,,,,,
Ciência da Computação,6,0,15,9,12
Engenharia da Computação,0,6,1,8,3


----------
* ### contagem relativa (%)

In [26]:
tab_cross_dif_jogos.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

uso de jogos impacta na visão sobre o curso?,1,2,3,4,5
curso,,,,,
Ciência da Computação,14.29%,0.00%,35.71%,21.43%,28.57%
Engenharia da Computação,0.00%,33.33%,5.56%,44.44%,16.67%


---------------
## Motivação x Expectativas

----------
* ### contagem absoluta

In [41]:
tab_cross_motiv_exp = pd.crosstab(vet_separados['motivação para entrar'], vet_separados['expectativas'])
tab_cross_motiv_exp

expectativas,Atividades extracurriculares,Bom networking,Capacitação e aprendizado,Curso com caráter acadêmico,Curso com caráter difícil,Curso com caráter matemático,Curso com caráter menos matemático,Curso com caráter mercadológico,Curso com caráter prático,Curso com caráter voltador para programação,Expectativas medianas,Expectativas positivas,Perfil de alunos entusiasmados,Perfil majoritariamente masculino,Projetos práticos
motivação para entrar,,,,,,,,,,,,,,,
Empregabilidade,0,0,5,0,0,1,0,2,0,0,0,1,0,1,1
Influência de Terceiros,0,0,2,0,1,0,0,0,0,0,1,0,0,0,0
Interesse por áreas do curso,2,2,23,1,2,1,1,2,3,1,0,4,2,1,0


----------
* ### contagem relativa (%)

In [42]:
tab_cross_motiv_exp.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

expectativas,Atividades extracurriculares,Bom networking,Capacitação e aprendizado,Curso com caráter acadêmico,Curso com caráter difícil,Curso com caráter matemático,Curso com caráter menos matemático,Curso com caráter mercadológico,Curso com caráter prático,Curso com caráter voltador para programação,Expectativas medianas,Expectativas positivas,Perfil de alunos entusiasmados,Perfil majoritariamente masculino,Projetos práticos
motivação para entrar,,,,,,,,,,,,,,,
Empregabilidade,0.00%,0.00%,45.45%,0.00%,0.00%,9.09%,0.00%,18.18%,0.00%,0.00%,0.00%,9.09%,0.00%,9.09%,9.09%
Influência de Terceiros,0.00%,0.00%,50.00%,0.00%,25.00%,0.00%,0.00%,0.00%,0.00%,0.00%,25.00%,0.00%,0.00%,0.00%,0.00%
Interesse por áreas do curso,4.44%,4.44%,51.11%,2.22%,4.44%,2.22%,2.22%,4.44%,6.67%,2.22%,0.00%,8.89%,4.44%,2.22%,0.00%


---------------
## Motivação x Mercado de Trabalho

----------
* ### contagem absoluta

In [43]:
tab_cross_motiv_merc = pd.crosstab(vet_separados['motivação para entrar'], vet_separados['mercado de trabalho'])
tab_cross_motiv_merc

mercado de trabalho,,Boa empregabilidade,Discrepância entre academia e mercado,Diversidade de oportunidades,Empregabilidade regional ruim,Expectativas negativas,Expectativas positivas,Mercado concorrido,Nenhuma,Socialmente Relevante,Trabalhar na área de interesse
motivação para entrar,,,,,,,,,,,
Empregabilidade,2,0,1,4,0,0,4,0,0,0,0
Influência de Terceiros,0,0,0,0,3,0,0,0,0,1,0
Interesse por áreas do curso,6,11,0,5,3,1,11,2,3,0,3


----------
* ### contagem relativa (%)

In [44]:
tab_cross_motiv_merc.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

mercado de trabalho,,Boa empregabilidade,Discrepância entre academia e mercado,Diversidade de oportunidades,Empregabilidade regional ruim,Expectativas negativas,Expectativas positivas,Mercado concorrido,Nenhuma,Socialmente Relevante,Trabalhar na área de interesse
motivação para entrar,,,,,,,,,,,
Empregabilidade,18.18%,0.00%,9.09%,36.36%,0.00%,0.00%,36.36%,0.00%,0.00%,0.00%,0.00%
Influência de Terceiros,0.00%,0.00%,0.00%,0.00%,75.00%,0.00%,0.00%,0.00%,0.00%,25.00%,0.00%
Interesse por áreas do curso,13.33%,24.44%,0.00%,11.11%,6.67%,2.22%,24.44%,4.44%,6.67%,0.00%,6.67%


---------------
## Motivação x Expectativas atendidas

----------
* ### contagem absoluta

In [45]:
tab_cross_motiv_atend = pd.crosstab(vet_separados['motivação para entrar'], vet_separados['expectativas atendidas'])
tab_cross_motiv_atend

expectativas atendidas,1,2,3,4,5
motivação para entrar,,,,,
Empregabilidade,1,5,0,5,0
Influência de Terceiros,0,0,1,3,0
Interesse por áreas do curso,3,4,16,19,3


----------
* ### contagem relativa (%)

In [46]:
tab_cross_motiv_atend.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

expectativas atendidas,1,2,3,4,5
motivação para entrar,,,,,
Empregabilidade,9.09%,45.45%,0.00%,45.45%,0.00%
Influência de Terceiros,0.00%,0.00%,25.00%,75.00%,0.00%
Interesse por áreas do curso,6.67%,8.89%,35.56%,42.22%,6.67%


---------------
## Motivação x Dificuldades

----------
* ### contagem absoluta

In [48]:
tab_cross_motiv_dif = pd.crosstab(vet_separados['motivação para entrar'], vet_separados['dificuldades'])
tab_cross_motiv_dif

dificuldades,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades de foco e organização pessoal,Dificuldades financeiras,Discrepância entre academia e mercado de trabalho,Estrutura do curso inflexível,Excesso de carga horária obrigatória,Falta de motivação pessoal,Horários conflitantes,Perfil de alunos do curso,Problemas na infraestrutura do campus
motivação para entrar,,,,,,,,,,,,
Empregabilidade,0,0,1,2,0,2,0,2,2,0,2,0
Influência de Terceiros,0,0,3,0,1,0,0,0,0,0,0,0
Interesse por áreas do curso,4,5,11,4,3,3,4,7,1,1,0,2


----------
* ### contagem relativa (%)

---------------
## Motivação x dificuldades podem te fazer desistir?

----------
* ### contagem absoluta

In [49]:
tab_cross_motiv_dif_giv = pd.crosstab(vet_separados['motivação para entrar'], vet_separados['dificuldades podem te fazer desistir?'])
tab_cross_motiv_dif_giv

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
motivação para entrar,,
Empregabilidade,3,8
Influência de Terceiros,4,0
Interesse por áreas do curso,27,18


----------
* ### contagem relativa (%)

In [50]:
tab_cross_motiv_dif_giv.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
motivação para entrar,,
Empregabilidade,27.27%,72.73%
Influência de Terceiros,100.00%,0.00%
Interesse por áreas do curso,60.00%,40.00%


In [51]:
tab_cross_motiv_dif.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

dificuldades,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades de foco e organização pessoal,Dificuldades financeiras,Discrepância entre academia e mercado de trabalho,Estrutura do curso inflexível,Excesso de carga horária obrigatória,Falta de motivação pessoal,Horários conflitantes,Perfil de alunos do curso,Problemas na infraestrutura do campus
motivação para entrar,,,,,,,,,,,,
Empregabilidade,0.00%,0.00%,9.09%,18.18%,0.00%,18.18%,0.00%,18.18%,18.18%,0.00%,18.18%,0.00%
Influência de Terceiros,0.00%,0.00%,75.00%,0.00%,25.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Interesse por áreas do curso,8.89%,11.11%,24.44%,8.89%,6.67%,6.67%,8.89%,15.56%,2.22%,2.22%,0.00%,4.44%


---------------
## Motivação x dificuldades podem te fazer desistir?

----------
* ### contagem absoluta

In [52]:
tab_cross_motiv_dif_giv = pd.crosstab(vet_separados['motivação para entrar'], vet_separados['dificuldades podem te fazer desistir?'])
tab_cross_motiv_dif_giv

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
motivação para entrar,,
Empregabilidade,3,8
Influência de Terceiros,4,0
Interesse por áreas do curso,27,18


----------
* ### contagem relativa (%)

In [53]:
tab_cross_motiv_dif_giv.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
motivação para entrar,,
Empregabilidade,27.27%,72.73%
Influência de Terceiros,100.00%,0.00%
Interesse por áreas do curso,60.00%,40.00%


---------------
## Motivação x QUAIS dificuldades podem te fazer desistir?

----------
* ### contagem absoluta

In [54]:
tab_cross_motiv_dif_giv_quais = pd.crosstab(vet_separados['motivação para entrar'], vet_separados['quais?'])
tab_cross_motiv_dif_giv_quais

quais?,,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades financeiras,Estrutura do curso inflexível,Falta de motivação pessoal,Horários conflitantes,Não sei
motivação para entrar,,,,,,,,,
Empregabilidade,3,0,0,4,1,0,2,1,0
Influência de Terceiros,4,0,0,0,0,0,0,0,0
Interesse por áreas do curso,31,2,1,2,3,4,0,1,1


----------
* ### contagem relativa (%)

In [55]:
tab_cross_motiv_dif_giv_quais.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

quais?,,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades financeiras,Estrutura do curso inflexível,Falta de motivação pessoal,Horários conflitantes,Não sei
motivação para entrar,,,,,,,,,
Empregabilidade,27.27%,0.00%,0.00%,36.36%,9.09%,0.00%,18.18%,9.09%,0.00%
Influência de Terceiros,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Interesse por áreas do curso,68.89%,4.44%,2.22%,4.44%,6.67%,8.89%,0.00%,2.22%,2.22%


---------------
## Motivação x dificuldades podem te fazer desistir?

----------
* ### contagem absoluta

In [56]:
tab_cross_motiv_dif_giv = pd.crosstab(vet_separados['motivação para entrar'], vet_separados['dificuldades podem te fazer desistir?'])
tab_cross_motiv_dif_giv

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
motivação para entrar,,
Empregabilidade,3,8
Influência de Terceiros,4,0
Interesse por áreas do curso,27,18


----------
* ### contagem relativa (%)

In [57]:
tab_cross_motiv_dif_giv.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
motivação para entrar,,
Empregabilidade,27.27%,72.73%
Influência de Terceiros,100.00%,0.00%
Interesse por áreas do curso,60.00%,40.00%


---------------
## Motivação x uso de jogos impacta na visão sobre o curso?

----------
* ### contagem absoluta

In [58]:
tab_cross_motiv_jogos = pd.crosstab(vet_separados['motivação para entrar'], vet_separados['uso de jogos impacta na visão sobre o curso?'])
tab_cross_motiv_jogos

uso de jogos impacta na visão sobre o curso?,1,2,3,4,5
motivação para entrar,,,,,
Empregabilidade,4,0,3,4,0
Influência de Terceiros,0,2,1,0,1
Interesse por áreas do curso,2,4,12,13,14


----------
* ### contagem relativa (%)

In [59]:
tab_cross_motiv_jogos.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

uso de jogos impacta na visão sobre o curso?,1,2,3,4,5
motivação para entrar,,,,,
Empregabilidade,36.36%,0.00%,27.27%,36.36%,0.00%
Influência de Terceiros,0.00%,50.00%,25.00%,0.00%,25.00%
Interesse por áreas do curso,4.44%,8.89%,26.67%,28.89%,31.11%


---------------
## Expectativas x Expectativas atendidas

----------
* ### contagem absoluta

In [60]:
tab_cross_expec_atend = pd.crosstab(vet_separados['expectativas'], vet_separados['expectativas atendidas'])
tab_cross_expec_atend

expectativas atendidas,1,2,3,4,5
expectativas,,,,,
Atividades extracurriculares,2,0,0,0,0
Bom networking,0,0,0,2,0
Capacitação e aprendizado,0,5,10,13,2
Curso com caráter acadêmico,0,0,0,0,1
Curso com caráter difícil,0,0,2,1,0
Curso com caráter matemático,0,0,0,2,0
Curso com caráter menos matemático,0,0,0,1,0
Curso com caráter mercadológico,0,0,0,4,0
Curso com caráter prático,0,2,1,0,0


----------
* ### contagem relativa (%)

In [61]:
tab_cross_expec_atend.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

expectativas atendidas,1,2,3,4,5
expectativas,,,,,
Atividades extracurriculares,100.00%,0.00%,0.00%,0.00%,0.00%
Bom networking,0.00%,0.00%,0.00%,100.00%,0.00%
Capacitação e aprendizado,0.00%,16.67%,33.33%,43.33%,6.67%
Curso com caráter acadêmico,0.00%,0.00%,0.00%,0.00%,100.00%
Curso com caráter difícil,0.00%,0.00%,66.67%,33.33%,0.00%
Curso com caráter matemático,0.00%,0.00%,0.00%,100.00%,0.00%
Curso com caráter menos matemático,0.00%,0.00%,0.00%,100.00%,0.00%
Curso com caráter mercadológico,0.00%,0.00%,0.00%,100.00%,0.00%
Curso com caráter prático,0.00%,66.67%,33.33%,0.00%,0.00%


---------------
## Expectativas x dificuldades

----------
* ### contagem absoluta

In [62]:
tab_cross_expec_dif = pd.crosstab(vet_separados['expectativas'], vet_separados['dificuldades'])
tab_cross_expec_dif

dificuldades,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades de foco e organização pessoal,Dificuldades financeiras,Discrepância entre academia e mercado de trabalho,Estrutura do curso inflexível,Excesso de carga horária obrigatória,Falta de motivação pessoal,Horários conflitantes,Perfil de alunos do curso,Problemas na infraestrutura do campus
expectativas,,,,,,,,,,,,
Atividades extracurriculares,0,0,0,0,0,0,1,1,0,0,0,0
Bom networking,0,0,1,0,1,0,0,0,0,0,0,0
Capacitação e aprendizado,2,2,9,2,2,1,2,4,3,1,2,0
Curso com caráter acadêmico,0,0,1,0,0,0,0,0,0,0,0,0
Curso com caráter difícil,0,1,1,0,0,0,0,0,0,0,0,1
Curso com caráter matemático,0,0,0,2,0,0,0,0,0,0,0,0
Curso com caráter menos matemático,1,0,0,0,0,0,0,0,0,0,0,0
Curso com caráter mercadológico,0,0,0,0,0,0,0,4,0,0,0,0
Curso com caráter prático,1,1,0,0,0,0,1,0,0,0,0,0


----------
* ### contagem relativa (%)

In [63]:
tab_cross_expec_dif.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

dificuldades,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades de foco e organização pessoal,Dificuldades financeiras,Discrepância entre academia e mercado de trabalho,Estrutura do curso inflexível,Excesso de carga horária obrigatória,Falta de motivação pessoal,Horários conflitantes,Perfil de alunos do curso,Problemas na infraestrutura do campus
expectativas,,,,,,,,,,,,
Atividades extracurriculares,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,50.00%,50.00%,0.00%,0.00%,0.00%,0.00%
Bom networking,0.00%,0.00%,50.00%,0.00%,50.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Capacitação e aprendizado,6.67%,6.67%,30.00%,6.67%,6.67%,3.33%,6.67%,13.33%,10.00%,3.33%,6.67%,0.00%
Curso com caráter acadêmico,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Curso com caráter difícil,0.00%,33.33%,33.33%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,33.33%
Curso com caráter matemático,0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Curso com caráter menos matemático,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
Curso com caráter mercadológico,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%
Curso com caráter prático,33.33%,33.33%,0.00%,0.00%,0.00%,0.00%,33.33%,0.00%,0.00%,0.00%,0.00%,0.00%


---------------
## Expectativas x dificuldades podem te fazer desistir?

----------
* ### contagem absoluta

In [64]:
tab_cross_expec_dif_giv = pd.crosstab(vet_separados['expectativas'], vet_separados['dificuldades podem te fazer desistir?'])
tab_cross_expec_dif_giv

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
expectativas,,
Atividades extracurriculares,0,2
Bom networking,2,0
Capacitação e aprendizado,16,14
Curso com caráter acadêmico,1,0
Curso com caráter difícil,3,0
Curso com caráter matemático,2,0
Curso com caráter menos matemático,1,0
Curso com caráter mercadológico,0,4
Curso com caráter prático,1,2


----------
* ### contagem relativa (%)

In [65]:
tab_cross_expec_dif_giv.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

dificuldades podem te fazer desistir?,"Não, de jeito nenhum!","Sim, com certeza!"
expectativas,,
Atividades extracurriculares,0.00%,100.00%
Bom networking,100.00%,0.00%
Capacitação e aprendizado,53.33%,46.67%
Curso com caráter acadêmico,100.00%,0.00%
Curso com caráter difícil,100.00%,0.00%
Curso com caráter matemático,100.00%,0.00%
Curso com caráter menos matemático,100.00%,0.00%
Curso com caráter mercadológico,0.00%,100.00%
Curso com caráter prático,33.33%,66.67%


---------------
## Expectativas x QUAIS dificuldades podem te fazer desistir?

----------
* ### contagem absoluta

In [67]:
tab_cross_expec_dif_giv_quais = pd.crosstab(vet_separados['expectativas'], vet_separados['quais?']).drop('', 1)
tab_cross_expec_dif_giv_quais = tab_cross_expec_dif_giv_quais[(tab_cross_expec_dif_giv_quais.T != 0).any()]
tab_cross_expec_dif_giv_quais.replace(0, "")

quais?,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades financeiras,Estrutura do curso inflexível,Falta de motivação pessoal,Horários conflitantes,Não sei
expectativas,,,,,,,,
Atividades extracurriculares,,,,,2,,,
Capacitação e aprendizado,2,1,3,1,2,2,,
Curso com caráter mercadológico,,,,2,,,2,
Curso com caráter prático,,,,,,,,1
Expectativas positivas,,,2,1,,,,
Projetos práticos,,,1,,,,,


----------
* ### contagem relativa (%)

In [68]:
tab_cross_expec_dif_giv_quais.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

quais?,Base pessoal de conhecimento ruim,Didática dos professores,Dificuldade e exigência das disciplinas,Dificuldades financeiras,Estrutura do curso inflexível,Falta de motivação pessoal,Horários conflitantes,Não sei
expectativas,,,,,,,,
Atividades extracurriculares,0.00%,0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%
Capacitação e aprendizado,18.18%,9.09%,27.27%,9.09%,18.18%,18.18%,0.00%,0.00%
Curso com caráter mercadológico,0.00%,0.00%,0.00%,50.00%,0.00%,0.00%,50.00%,0.00%
Curso com caráter prático,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
Expectativas positivas,0.00%,0.00%,66.67%,33.33%,0.00%,0.00%,0.00%,0.00%
Projetos práticos,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%


---------------
## Expectativas x uso de jogos impacta na visão sobre o curso?

----------
* ### contagem absoluta

In [69]:
tab_cross_expec_jogos = pd.crosstab(vet_separados['expectativas'], vet_separados['uso de jogos impacta na visão sobre o curso?'])
tab_cross_expec_jogos

uso de jogos impacta na visão sobre o curso?,1,2,3,4,5
expectativas,,,,,
Atividades extracurriculares,0,0,0,2,0
Bom networking,0,0,0,0,2
Capacitação e aprendizado,6,5,8,7,4
Curso com caráter acadêmico,0,0,0,0,1
Curso com caráter difícil,0,1,0,0,2
Curso com caráter matemático,0,0,0,2,0
Curso com caráter menos matemático,0,0,1,0,0
Curso com caráter mercadológico,0,0,4,0,0
Curso com caráter prático,0,0,0,0,3


----------
* ### contagem relativa (%)

In [70]:
tab_cross_expec_jogos.apply(lambda r: r/r.sum(), axis=1).style.format("{:.2%}")

uso de jogos impacta na visão sobre o curso?,1,2,3,4,5
expectativas,,,,,
Atividades extracurriculares,0.00%,0.00%,0.00%,100.00%,0.00%
Bom networking,0.00%,0.00%,0.00%,0.00%,100.00%
Capacitação e aprendizado,20.00%,16.67%,26.67%,23.33%,13.33%
Curso com caráter acadêmico,0.00%,0.00%,0.00%,0.00%,100.00%
Curso com caráter difícil,0.00%,33.33%,0.00%,0.00%,66.67%
Curso com caráter matemático,0.00%,0.00%,0.00%,100.00%,0.00%
Curso com caráter menos matemático,0.00%,0.00%,100.00%,0.00%,0.00%
Curso com caráter mercadológico,0.00%,0.00%,100.00%,0.00%,0.00%
Curso com caráter prático,0.00%,0.00%,0.00%,0.00%,100.00%


----------
## Variáveis auxiliares
----------

In [8]:
# simplifying the questions

map_col_vet={'Qual é a sua idade?': 'idade', \
             'Qual é o seu gênero?': 'gênero', \
             'De qual curso você é aluno?': 'curso', \
             'Em que instituição você cursa?': 'instituição', \
             'Em qual ano e semestre você entrou no curso?': 'semestre entrada', \
             'Qual foi a sua motivação ao entrar no curso?': 'motivação para entrar', \
             'Ao ingressar, quais eram as suas expectativas sobre o curso?': 'expectativas', \
             'Quais eram as suas perspectivas em relação ao mercado de trabalho?': 'mercado de trabalho', \
             'Até este momento do curso, o quanto você considera que suas expectativas vêm sendo atendidas ou frustradas?':'expectativas atendidas', \
             'Quais são as principais dificuldades que você vem encontrando no decorrer do curso?':'dificuldades', \
             'Você acha que alguma(s) dessas dificuldade(s) pode(m) lhe fazer desistir do curso?':'dificuldades podem te fazer desistir?',\
             'Se sim, quais, por exemplo?':'quais?', \
             'O quanto você considera que o uso de métodos de ensino mais lúdicos durante as disciplinas do curso (como jogos, por exemplo) podem impactar sua visão sobre o curso?': 'uso de jogos impacta na visão sobre o curso?' \
            }

# replacing the codes by the categorie names
codesByCat = {
    "motivação para entrar" : {"IntAC":"Interesse por áreas do curso",\
             "Emp": "Empregabilidade",\
             "IntTer":"Influência de Terceiros"\
            },

    "expectativas" : {
        "CCMat": "Curso com caráter matemático", \
        "PAMasc": "Perfil majoritariamente masculino", \
        "CCDif":"Curso com caráter difícil", \
        "PAEnt":"Perfil de alunos entusiasmados",\
        "CCPrat": "Curso com caráter prático", \
        "CapAp": "Capacitação e aprendizado", \
        "EP": "Expectativas positivas", \
        "CCProg": "Curso com caráter voltador para programação",\
        "AtivExtCur": "Atividades extracurriculares",\
        "AtivEC": "Atividades extracurriculares",\
        "CCMerc":"Curso com caráter mercadológico",\
        "CCAcad": "Curso com caráter acadêmico", \
        "CCMenMat": "Curso com caráter menos matemático",\
        "Netw": "Bom networking",\
        "ProjPrat":"Projetos práticos",\
        "EM": "Expectativas medianas"\
        },

    "mercado de trabalho" : {
        "BEmp": "Boa empregabilidade", \
        "TrabAI": "Trabalhar na área de interesse", \
        "EP":"Expectativas positivas", \
        "N":"Nenhuma",\
        "DivOp": "Diversidade de oportunidades", \
        "EmpRR": "Empregabilidade regional ruim", \
        "SocRel": "Socialmente Relevante", \
        "MercC": "Mercado concorrido",\
        "EN": "Expectativas negativas",\
        "DiscAM": "Discrepância entre academia e mercado"\
        },

    "dificuldades" : {
        "DOrgP": "Dificuldades de foco e organização pessoal", \
        "DidProf": "Didática dos professores", \
        "ProbInf":"Problemas na infraestrutura do campus", \
        "EstrCInf": "Estrutura do curso inflexível",\
        "DifDisc": "Dificuldade e exigência das disciplinas",\
        "BConR": "Base pessoal de conhecimento ruim",\
        "ExCHO": "Excesso de carga horária obrigatória",\
        "DiscAM": "Discrepância entre academia e mercado de trabalho",\
        "DifFin": "Dificuldades financeiras",\
        "HorConf": "Horários conflitantes",\
        "FMotivP": "Falta de motivação pessoal",\
        "PerfAlu": "Perfil de alunos do curso"\
        },

    "quais?" : {
        "DidProf": "Didática dos professores", \
        "EstrCInf": "Estrutura do curso inflexível",\
        "DifDisc": "Dificuldade e exigência das disciplinas",\
        "BConR": "Base pessoal de conhecimento ruim",\
        "DifFin": "Dificuldades financeiras",\
        "HorConf": "Horários conflitantes",\
        "N": "Não sei",\
        "FMotP": "Falta de motivação pessoal",\
        }
}